In [141]:
import argparse
import subprocess
import torch
import torchvision
import os
import torch.optim
import tqdm
# import apex.amp as amp
import time
import json
import pprint
import torch.nn.functional as F


In [142]:
annotations_file = 'inter/annotations/naju_train_update.json'
category_name = 'person'
with open(annotations_file, 'r') as f:
    data = json.load(f)
    print(data.keys())
    for c in data['categories'] :
        print(c['name'])

    cat = [c for c in data['categories']][0]

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
person


In [5]:
from coco_assistant import COCO_Assistant

In [6]:
os.getcwd()

'/home/beomgon/dataset'

In [12]:
img_dir = os.path.join('/home/beomgon/dataset/coco_inter', 'images')
ann_dir = os.path.join('/home/beomgon/dataset/coco_inter', 'annotations')
img_dir

'/home/beomgon/dataset/coco_inter/images'

In [13]:
cas = COCO_Assistant(img_dir, ann_dir)

loading annotations into memory...
Done (t=4.69s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [ ]:
cas.merge()

Merging image dirs


  0%|                                                                          | 0/2 [00:00<?, ?it/s]

In [170]:
annotations_file = 'inter/annotations/naju_train_update.json'
category_name = 'person'
with open(annotations_file, 'r') as f:
    data = json.load(f)
    print(data.keys())
    for c in data['categories'] :
        print(c['name'])

    cat = [c for c in data['categories']][0]

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
person


In [171]:
data['images'][0]

{'id': 0,
 'width': 848,
 'height': 480,
 'file_name': 'MANY01_4_5_86.jpg',
 'license': 1,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': ''}

In [172]:
data['annotations'][0]

{'id': 0,
 'image_id': 0,
 'category_id': 1,
 'segmentation': [],
 'area': 30780,
 'bbox': [656, 129, 190, 162],
 'iscrowd': 0,
 'keypoints': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  843,
  161,
  2,
  790,
  171,
  2,
  756,
  167,
  2],
 'num_keypoints': 3}

In [173]:
train_image_list = []
for image in data['images'] :
    if image['id'] % 20 == 0 :
        train_image_list.append(image)
        
train_anno_list = []
for anno in data['annotations'] :
    if anno['image_id'] % 20 == 0 :
        train_anno_list.append(anno)

In [174]:
# (train_anno_list)[:2]

In [175]:
len(train_image_list)

5119

In [176]:
for i, image in enumerate(train_image_list) :
    # print(i)
    train_image_list[i]['id'] = int(image['id'] / 20)
    
for i, anno in enumerate(train_anno_list) :
    # print(i)
    train_anno_list[i]['image_id'] = int(anno['image_id'] / 20)    

for i, anno in enumerate(train_anno_list) :
    # print(i)
    train_anno_list[i]['id'] = i  

In [177]:
len(train_anno_list)

10491

In [178]:
data['images'] = train_image_list
data['annotations'] = train_anno_list

In [179]:
data['images'][-1]

{'id': 5118,
 'width': 848,
 'height': 480,
 'file_name': '2205269_8_180.jpg',
 'license': 1,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': ''}

In [180]:
data['annotations'][-1]

{'id': 10490,
 'image_id': 5118,
 'category_id': 1,
 'segmentation': [],
 'area': 29295,
 'bbox': [138, 2, 189, 155],
 'iscrowd': 1,
 'keypoints': [171,
  81,
  2,
  177,
  40,
  2,
  196,
  5,
  2,
  177,
  5,
  2,
  266,
  3,
  2,
  305,
  17,
  2,
  296,
  59,
  2],
 'num_keypoints': 7}

In [181]:
print(len(data['images']))
print(len(data['annotations']))

5119
10491


In [182]:
# file_name_list = []
# for image in data['images'] :
#     file_name_list.append(image['file_name'])
    

In [115]:
# import shutil

# for file in file_name_list :
#     src = '/home/beomgon/dataset/inter/images/' + file
#     dst = '/home/beomgon/dataset/reduced_inter/images/train/' + file
#     shutil.copyfile(src, dst)
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/beomgon/dataset/inter/images/2205268_6_105.jpg'

In [1]:
import shutil
import argparse
import subprocess
import os
import tqdm
# import apex.amp as amp
import time
import json
import pprint

class ReducedDataAnno() :
    def __init__(self, annfile, src_dir, dst_dir, new_anno, dev=20):
        self.annfile = annfile
        self.dev = dev
        self.category_name = 'person'
        self.image_list = []
        self.anno_list = []
        self.file_name_list = []
        
        self.src_dir = src_dir
        self.dst_dir = dst_dir
        self.new_anno = new_anno

        with open(self.annfile, 'r') as f:
            self.data = json.load(f)
            print(self.data.keys())
            for c in self.data['categories'] :
                print(c['name'])

            cat = [c for c in self.data['categories']][0]   
            
    def get_anno(self) :
        
        for image in self.data['images'] :
            if image['id'] % self.dev == 0 :
                self.image_list.append(image)

        for anno in self.data['annotations'] :
            if anno['id'] % self.dev == 0 :
                self.anno_list.append(anno)    

        for i, image in enumerate(self.image_list) :
            # print(i)
            self.image_list[i]['id'] = int(image['id'] / self.dev)

        for i, anno in enumerate(self.anno_list) :
            # print(i)
            self.anno_list[i]['image_id'] = int(anno['image_id'] / self.dev)    

        for i, anno in enumerate(self.anno_list) :
            # print(i)
            self.anno_list[i]['id'] = i              

        self.data['images'] = self.image_list
        self.data['annotations'] = self.anno_list   
        
        with open(self.new_anno, 'w') as outfile:
            json.dump(self.data, outfile)        
        
        
    def copy_files(self) :

        for image in self.data['images'] :
            self.file_name_list.append(image['file_name'])    
            
        for file in self.file_name_list :
            src = self.src_dir + file
            dst = self.dst_dir + file
            shutil.copyfile(src, dst)      
            
    def __call__(self):
        self.get_anno()
        self.copy_files()
        

        
        
        

In [3]:
# naju train
annfile = 'inter/annotations/naju_train_update.json'
src_dir = '/home/beomgon/dataset/inter/images/'
dst_dir = '/home/beomgon/dataset/reduced_inter/naju_images/train/'
new_anno = '/home/beomgon/dataset/reduced_inter/annotations/naju_train.json'
reduced_train = ReducedDataAnno(annfile, src_dir, dst_dir, new_anno)
reduced_train()

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
person


In [4]:
# naju test
annfile = 'inter/annotations/naju_test_update.json'
src_dir = '/home/beomgon/dataset/inter/images/'
dst_dir = '/home/beomgon/dataset/reduced_inter/naju_images/val/'
new_anno = '/home/beomgon/dataset/reduced_inter/annotations/naju_val.json'
reduced_train = ReducedDataAnno(annfile, src_dir, dst_dir, new_anno)
reduced_train()


dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
person


In [5]:
# showroom train
annfile = 'inter/annotations/showroom_0825_train.json'
src_dir = '/home/beomgon/dataset/inter/images/'
dst_dir = '/home/beomgon/dataset/reduced_inter/showroom_images/train/'
new_anno = '/home/beomgon/dataset/reduced_inter/annotations/showroom_train.json'
reduced_train = ReducedDataAnno(annfile, src_dir, dst_dir, new_anno)
reduced_train()

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
InterMinds 7 Point.


In [6]:
# showroom test
annfile = 'inter/annotations/showroom_0825_test.json'
src_dir = '/home/beomgon/dataset/inter/images/'
dst_dir = '/home/beomgon/dataset/reduced_inter/showroom_images/val/'
new_anno = '/home/beomgon/dataset/reduced_inter/annotations/showroom_val.json'
reduced_train = ReducedDataAnno(annfile, src_dir, dst_dir, new_anno)
reduced_train()

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
InterMinds 7 Point.


In [8]:
from coco_assistant import COCO_Assistant

In [9]:
# merge train data
train_dir = '/home/beomgon/dataset/total/train/'
train_img_dir = train_dir + 'images'
train_ann_dir = train_dir + 'annotations'
cas = COCO_Assistant(train_img_dir, train_ann_dir)
cas.merge()

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=1.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Merging image dirs


100%|██████████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 21.50s/it]


Merging annotations


100%|██████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.54it/s]


In [131]:
train_img_dir

'/home/beomgon/dataset/total/train/images'

In [132]:
train_ann_dir

'/home/beomgon/dataset/total/train/annotations'

In [11]:
# merge test data
val_dir = '/home/beomgon/dataset/total/val/'
val_img_dir = val_dir + 'images'
val_ann_dir = val_dir + 'annotations'
cas = COCO_Assistant(val_img_dir, val_ann_dir)
cas.merge()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Merging image dirs


100%|██████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.04it/s]


Merging annotations


100%|██████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 83.45it/s]
